In [1]:
include("../src/extractdata.jl")
include("../src/scrap.jl")

get_tag

In [2]:
en_words = unique!(vcat(first.(en_triples), last.(en_triples)))
enSet = Set(en_words)
zh_words = unique!(vcat(first.(zh_triples), last.(zh_triples)))
zhSet = Set(zh_words);

In [3]:
txts = split(strip(read(open("ILLs/wiki_ills_zh2en.txt", "r"), String)), '\n')
MT_ILLs_zh2en = [(zh, replace(en, ' '=>'_')) for (zh, en) in split.(txts, '\t')]
MT_ILLs_valid1 = filter(i->i[2] ∈ enSet, MT_ILLs_zh2en)

37558-element Vector{Tuple{SubString{String}, String}}:
 ("四川省", "Sichuan")
 ("王菲", "Faye_Wong")
 ("袁世凯", "Yuan_Shikai")
 ("勒内·笛卡尔", "René_Descartes")
 ("克洛泰尔一世", "Chlothar_I")
 ("聖尼古拉球場", "Stadio_San_Nicola")
 ("阮慶", "Nguyễn_Khánh")
 ("陳文香", "Trần_Văn_Hương")
 ("黃晉發", "Huỳnh_Tấn_Phát")
 ("南越南共和國", "Provisional_Revolutionary_Government_of_the_Republic_of_South_Vietnam")
 ("沙查·巴隆·科恩", "Sacha_Baron_Cohen")
 ("海斯廷斯", "Hastings")
 ("洛斯雷亚莱霍斯", "Los_Realejos")
 ⋮
 ("拉努斯", "Lanús")
 ("日本棋院", "Nihon_Ki-in")
 ("加拿大改革黨", "Reform_Party_of_Canada")
 ("自治区_(墨西哥)", "Municipalities_of_Mexico")
 ("溫莎大學", "University_of_Windsor")
 ("俄羅斯21歲以下國家足球隊", "Russia_national_under-21_football_team")
 ("加蓬总理", "Prime_Minister_of_Gabon")
 ("科摩罗总统", "List_of_heads_of_state_of_the_Comoros")
 ("数码宝贝系列电子游戏列表", "List_of_Digimon_video_games")
 ("巴登", "Baden")
 ("季米特里斯·库楚巴斯", "Dimitris_Koutsoumpas")
 ("KILL_la_KILL", "Kill_la_Kill")

In [4]:
txts = split(strip(read(open("ILLs/wiki_ills_en2zh.txt", "r"), String)), '\n')
MT_ILLs_en2zh = [(en, replace(zh, ' '=>'_')) for (en, zh) in split.(txts, '\t')]
MT_ILLs_valid2 = reverse.(filter(i->i[2] ∈ zhSet, MT_ILLs_en2zh))

34713-element Vector{Tuple{String, SubString{String}}}:
 ("基督教穆斯林民主力量党", "Lakas–CMD")
 ("朱善璐", "Zhu_Shanlu")
 ("潮汕地區", "Chaoshan")
 ("薩蘭縣", "Saran_district")
 ("卡尔瓦略", "Carballo")
 ("白俄罗斯共和国国歌", "My_Belarusy")
 ("克勞斯·阿洛夫斯", "Klaus_Allofs")
 ("阿尔盖罗", "Alghero")
 ("西拉法葉", "West_Lafayette,_Indiana")
 ("薩道義", "Ernest_Mason_Satow")
 ("阿菲永卡拉希萨尔", "Afyonkarahisar")
 ("斯坦利·鲍德温", "Stanley_Baldwin")
 ("色萨利", "Thessaly")
 ⋮
 ("杨诗曼", "Agnez_Mo")
 ("弗里德蘭戰役", "Battle_of_Friedland")
 ("金盞花屬", "Calendula")
 ("黛菲", "Duffy_(singer)")
 ("巴黎第九大学", "Paris_Dauphine_University")
 ("加州州立大學洛杉磯分校", "California_State_University,_Los_Angeles")
 ("桑坦德銀行", "Banco_Santander")
 ("阿尔托大学", "Aalto_University")
 ("葛麗絲·霍普", "Grace_Hopper")
 ("彭震海", "Pang_Chun-hoi")
 ("德克薩斯理工大學", "Texas_Tech_University")
 ("温尼马卡_(内华达州)", "Winnemucca,_Nevada")

In [2]:
filter(i->i[1]==i[2], ILLs)

Tuple{String, String}[]

In [9]:
MT_ILLs_valid = filter(i->i[1]!=i[2], union(MT_ILLs_valid2, MT_ILLs_valid1, reverse.(ILLs)))

42115-element Vector{Tuple{AbstractString, AbstractString}}:
 ("基督教穆斯林民主力量党", "Lakas–CMD")
 ("朱善璐", "Zhu_Shanlu")
 ("潮汕地區", "Chaoshan")
 ("薩蘭縣", "Saran_district")
 ("卡尔瓦略", "Carballo")
 ("白俄罗斯共和国国歌", "My_Belarusy")
 ("克勞斯·阿洛夫斯", "Klaus_Allofs")
 ("阿尔盖罗", "Alghero")
 ("西拉法葉", "West_Lafayette,_Indiana")
 ("薩道義", "Ernest_Mason_Satow")
 ("阿菲永卡拉希萨尔", "Afyonkarahisar")
 ("斯坦利·鲍德温", "Stanley_Baldwin")
 ("色萨利", "Thessaly")
 ⋮
 ("天津权健足球俱乐部", "Tianjin_Quanjian_F.C.")
 ("约翰·克里", "John_Kerry")
 ("莫拉德夫雷", "Móra_d'Ebre")
 ("卡諾", "Kano")
 ("成都钱宝足球俱乐部", "Chengdu_Qianbao_F.C.")
 ("西里尔·内维尔，第一代内维尔男爵", "Cyril_Newall,_1st_Baron_Newall")
 ("占碑市", "Jambi_City")
 ("巴讷费尔德", "Barneveld")
 ("美国联邦最高法院大法官列表", "List_of_Justices_of_the_Supreme_Court_of_the_United_States")
 ("新界西選區", "New_Territories_West_(constituency)")
 ("九龍西選區", "Kowloon_West_(constituency)")
 ("列昂尼德·伊里奇·勃列日涅夫", "Leonid_Brezhnev")

In [10]:
open("ILLs/ILLs_valid.txt", "w") do io
    for (zh, en) in MT_ILLs_valid
        println(io, en,'\t', zh)
    end
end